# Audience Demo

## Import packages and data

In [81]:
import sys
sys.path.insert(1,'/Users/davidbartram-shaw/3D/Sponsor_Feature_Project')

import pandas as pd
import numpy as np
import time
import pickle
import multiprocessing
from joblib import Parallel, delayed

In [82]:
# Cards
test_cards = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/sample_data/test_cards.csv', header=0, sep=',', encoding='utf-8')
test_cards['Name']=test_cards.Name.str.strip()

# Site data
site_visits = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/sample_data/site_visits.csv', header=0, sep=',', encoding='utf-8')
site_features_customer_level = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/sample_data/site_features_customer_level.csv', header=0, sep=',', encoding='utf-8')
site_features_audience_level = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/sample_data/site_features_audience_level.csv', header=0, sep=',', encoding='utf-8')
req_cols = ['CARD_KEY', 'BIKES', 'CARS', 'FASHION', 'FINANCE', 'FOOD','GARDENING', 'MUSIC', 'PARENTS',  'SPORT', 'TECH','TRAVEL', 'HOUSE', 'ENTERTAINMENT','Name']

site_visits = pd.merge(site_visits,test_cards, left_on='CARD_KEY', right_on='card_key').sort_values('CATEGORY_COUNT',ascending=False)
site_features_customer_level = pd.merge(site_features_customer_level,test_cards, left_on='CARD_KEY', right_on='card_key')[req_cols]
site_features_audience_level = pd.merge(site_features_audience_level,test_cards, left_on='CARD_KEY', right_on='card_key')[req_cols]

# Trans data
trans_features_customer_level_by_partner = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/sample_data/trans_features_customer_level_by_partner.csv', header=0, sep=',', encoding='utf-8')
trans_features_audience_level = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/sample_data/trans_features_audience_level.csv', header=0, sep=',', encoding='utf-8').sort_values('SPORT',ascending=False)
trans_features_customer_level = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/sample_data/trans_features_customer_level.csv', header=0, sep=',', encoding='utf-8')

trans_features_customer_level_by_partner = pd.merge(trans_features_customer_level_by_partner,test_cards, left_on='CARD_KEY', right_on='card_key').sort_values('TRANSACTIONS',ascending=False)
trans_features_customer_level = pd.merge(trans_features_customer_level,test_cards, left_on='CARD_KEY', right_on='card_key')
trans_features_audience_level = pd.merge(trans_features_audience_level,test_cards, left_on='CARD_KEY', right_on='card_key')

trans_features_customer_level_by_partner = trans_features_customer_level_by_partner[trans_features_customer_level_by_partner.PARTNER!='PRUHEALTH']
trans_features_customer_level_by_partner = trans_features_customer_level_by_partner[trans_features_customer_level_by_partner.PARTNER!='ESTORES']

del trans_features_customer_level_by_partner['NORMED_TRANS']
del trans_features_customer_level_by_partner['card_key']
del trans_features_customer_level_by_partner['INDEX_NUMBER']
del trans_features_customer_level_by_partner['PETS']



## Final partner level data

In [ ]:
# PARTNER LEVEL
partner_category_similarity_df = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/pipeline_data/final_partner_vectors.csv')
del partner_category_similarity_df['Unnamed: 0']


In [ ]:
# Extract Text & clean function
input_save_name='partner_final_full'
partner_corpus = clean_text_options(partner_corpus
                                  ,input_data_save_name = input_save_name
                                  ,clean_text='yes'
                                  ,retain_only_nouns='yes'
                                  ,retain_english_words='yes'
                                  ,remove_on_tfidf='yes'
                                  ,tfidf_cut_off=0.6
                                  ,tfidf_binary=False
                                  ,tfidf_sublinear_tf=False
                                  ,norm=None
                                  )


## Make jupyter full width

In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

##### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Alex J example
##### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Each Partner shopped with and their respective category_vector

In [83]:
aj = trans_features_customer_level_by_partner[trans_features_customer_level_by_partner.Name=='Alex J']
aj

,BIKES,CARS,FASHION,FINANCE,FOOD,GARDENING,MUSIC,PARENTS,SPORT,TECHNOLOGY,TRAVEL,HOME,ENTERTAINMENT,CARD_KEY,PARTNER,TRANSACTIONS,Name
83,0.0,0.000000,0.027060,0.14436,0.0000,0.0000,0.006780,0.004500,0.000000,0.0609,0.056400,0.00000,0.000000,51878664,AMERICANEX,6,Alex J
80,0.0,0.010425,0.010425,0.05730,0.0156,0.0000,0.010425,0.046875,0.026025,0.3333,0.156225,0.03645,0.046875,51878664,BP,3,Alex J
90,0.0,0.000000,0.078400,0.37260,0.4510,0.0000,0.039200,0.058800,0.000000,0.7254,0.000000,0.19600,0.078400,51878664,AO.COM,2,Alex J
88,0.0,0.000000,0.000000,0.00000,0.0000,0.0000,0.000000,0.000000,0.000000,1.0000,0.000000,0.00000,0.000000,51878664,TOOLBAR,2,Alex J
81,0.0,0.028800,0.000000,0.22120,0.0000,0.0000,0.038500,0.105800,0.028800,0.1058,0.384600,0.05770,0.028800,51878664,VIRGINTR,1,Alex J
82,0.0,0.042300,0.000000,0.22540,0.0000,0.0563,0.042300,0.042300,0.000000,0.0563,0.408500,0.11270,0.014100,51878664,EXPEDIA,1,Alex J
86,0.0,0.000000,0.000000,0.00000,1.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,51878664,PIZZA EXPRESS,1,Alex J
87,0.0,0.000000,0.000000,0.00000,1.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,51878664,CAFFE NERO,1,Alex J
89,0.0,0.000000,0.270800,0.14580,0.0417,0.0417,0.000000,0.048600,0.013900,0.0278,0.062500,0.13190,0.215300,51878664,HOUSE OF FRASER,1,Alex J
91,0.0,0.000000,0.094100,0.01180,0.0000,0.0000,0.023500,0.094100,0.058800,0.2941,0.000000,0.34120,0.082400,51878664,PC WORLD,1,Alex J


In [84]:
#http://pandas.pydata.org/pandas-docs/stable/style.html
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
aj_formated = aj.style.background_gradient(cmap=cm)
aj_formated

## Customer level transactional vector
combining all partner vectors - Normalising on a CUSTOMER level

In [85]:
aj_customer_level = trans_features_customer_level[trans_features_customer_level.Name=='Alex J']
aj_customer_level

,CARD_KEY,BIKES,CARS,FASHION,FINANCE,FOOD,GARDENING,MUSIC,PARENTS,PETS,SPORT,TECHNOLOGY,TRAVEL,HOME,ENTERTAINMENT,Name,card_key
9,51878664,0.0,0.0226,0.1334,0.9437,0.8811,0.0272,0.0446,0.1113,0,0.0354,1.0,0.2964,0.3356,0.4068,Alex J,51878664


## Audience level transactional vector
combining all partner vectors - Normalising on an Audience level

In [86]:
aj_audience_level = trans_features_audience_level[trans_features_audience_level.Name=='Alex J']
aj_audience_level

,CARD_KEY,BIKES,CARS,FASHION,FINANCE,FOOD,GARDENING,MUSIC,PARENTS,PETS,SPORT,TECHNOLOGY,TRAVEL,HOME,ENTERTAINMENT,Name,card_key
7,51878664,0.0,0.0127,0.4411,1.0,1.0,0.0365,0.4021,0.3333,0,0.1441,0.477,0.1618,0.6463,1.0,Alex J,51878664


## Categorised Site Visits per customer

In [87]:
aj_site_visits = site_visits[site_visits.Name=='Alex J']
aj_site_visits

,CARD_KEY,CARD_HASH_ID,PREDICTED_LABEL,CATEGORY_COUNT,Name,card_key
45,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,travel,28,Alex J,51878664
42,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,news,23,Alex J,51878664
50,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,food,8,Alex J,51878664
49,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,music,8,Alex J,51878664
43,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,sport,5,Alex J,51878664
51,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,weather,5,Alex J,51878664
46,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,finance,4,Alex J,51878664
44,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,cars,3,Alex J,51878664
48,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,general,3,Alex J,51878664
47,51878664,8329b2eb2fe4ed6eb4ad44e61a6702d035d3e40c,tech,2,Alex J,51878664


## Customer level Site vector
combining all partner vectors - Normalising on an Audience level

In [88]:
aj_site_customer_view = site_features_customer_level[site_features_customer_level.Name=='Alex J']
aj_site_customer_view

,CARD_KEY,BIKES,CARS,FASHION,FINANCE,FOOD,GARDENING,MUSIC,PARENTS,SPORT,TECH,TRAVEL,HOUSE,ENTERTAINMENT,Name
8,51878664,0,0.124547,0,0.130478,0.30216,0,0.184514,0.0,0.203434,0.02916,0.947174,0.0,0.0,Alex J


## Example Use-Case: Best Fashion Audience


In [89]:
fashion_audience = trans_features_audience_level.sort_values('FASHION',ascending=False).style.background_gradient(cmap=cm)
fashion_audience

# Transactional Category Vectors

In [90]:
partner_category_similarity_df = pd.read_csv('/Users/davidbartram-shaw/3D/Sponsor_Feature_Project/data/pipeline_data/final_partner_vectors.csv')
del partner_category_similarity_df['Unnamed: 0']
##partner_category_similarity_df

In [91]:
partner_category_similarity_df[partner_category_similarity_df.method=='web_text_classify'].sort_values('sport',ascending=False).head(10).style.background_gradient(cmap=cm)